In [1]:
print("hi")

hi


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Jupyter") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.cores", "2") \
    .config("spark.cores.max", "4") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

df = spark.read.csv("/opt/spark-data/input/data.csv", header=True, inferSchema=True)
df.show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/15 17:58:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+-------+---+------+
|   name|age|salary|
+-------+---+------+
| manish| 28| 55000|
| ramesh| 32| 48000|
|  rohan| 26| 61000|
|  sapna| 30| 72000|
|shivani| 27| 68000|
+-------+---+------+



In [3]:
spark

In [4]:
from pyspark.sql import SparkSession
from faker import Faker
import random


fake = Faker()

# Create skewed customer data (80% customers in one city)
customers_data = []
for i in range(1000000):
    city = "New York" if random.random() < 0.8 else fake.city()
    customers_data.append((i, fake.name(), city, fake.email()))

customers_df = spark.createDataFrame(
    customers_data, 
    ["customer_id", "name", "city", "email"]
)

# Create orders data
orders_data = [(i, random.randint(0, 999999), random.randint(10, 1000)) 
               for i in range(5000000)]
orders_df = spark.createDataFrame(
    orders_data, 
    ["order_id", "customer_id", "amount"]
)

# BAD: This will cause massive skew (one partition processes 80% of data)
result = orders_df.join(customers_df, "customer_id")
result.write.mode("overwrite").parquet("/tmp/skewed_output")

25/12/15 18:01:16 WARN TaskSetManager: Stage 3 contains a task of very large size (17749 KiB). The maximum recommended task size is 1000 KiB.
25/12/15 18:01:20 WARN TaskSetManager: Stage 4 contains a task of very large size (12990 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

In [5]:
display(orders_df)

DataFrame[order_id: bigint, customer_id: bigint, amount: bigint]